# RAG Process
- OpenAI API Key 준비

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### 1. 필요한 라이브러리 설치 및 임포트
- 목표 정의 및 데이터 준비

In [18]:
# 필요한 라이브러리 import
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.runnables import RunnableMap, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
import re

### 2. PDF 파일 로딩 및 텍스트 추출

In [3]:
# 텍스트 파일을 문서로 로드
loader1 = PyPDFLoader("../data/squat1.pdf")  # 만약 파일의 이미지까지 불러오고 싶다면 extract_images=True 추가
doc1 = loader1.load()

loader2 = PyPDFLoader("../data/squat2.pdf")
doc2 = loader2.load()

loader3 = PyPDFLoader("../data/squat3.pdf")
doc3 = loader3.load()

### 3. 문서 조각화 (Chunking)

In [4]:
# 문서 분할 클래스 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=30,
    separators=["\n\n", "\n", ".", " ", ""]
    )
split_docs = text_splitter.split_documents(doc1 + doc2 + doc3)

### 4. 임베딩 모델 설정 및 ChromaDB에 임베딩 저장

In [5]:
# OpenAI Embeddings()
embedding_model = OpenAIEmbeddings()

# Chroma 벡터 저장소에 문서 임베딩 저장
vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    persist_directory="../chroma_db/pdf_docs")

# 로컬에 저장
vectorstore.persist()

C:\Users\user\AppData\Local\Temp\ipykernel_3352\2081955093.py:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


### 5. retirever 검색기 설정

In [6]:
# 저장된 Chroma 벡터 DB 로드
vectorstore = Chroma(
    persist_directory="../chroma_db/pdf_docs",
    embedding_function=embedding_model
    )

# Retriever 생성
retriever = vectorstore.as_retriever(
    search_type="similarity",           # 유사도 분석 기반
    search_kwargs={"k": 5}              # 3개의 문장만 갖고오기
    )

C:\Users\user\AppData\Local\Temp\ipykernel_3352\1314364453.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


### 6. Prompt 설정

In [16]:
# 프롬포트 설정하기
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
당신은 재활 의학 전문의 면허를 소지하였고, 물리치료 및 도수치료 전문가 입니다.  
다음 문서 내용과 사전에 습득된 지식을 바탕으로 사용자의 질문에 대해 충실하고 근거 있는 답변을 작성해 주세요.

<문서 정보>
{context}

<질문>
{question}

<답변>
""".strip()
)

### 7. LLM 설정

In [19]:
# OpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)          # temperature: 창의성 레벨 (0~1)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def format_response(text: str) -> str:
    # 마침표, 물음표, 느낌표 뒤에 줄바꿈 추가
    lines = re.sub(r'([.!?])\s+', r'\1\n', text.strip())
    return lines

### 8. RAG Chain 설정

In [20]:
# chain 연결하기
rag_chain = (
    RunnableMap({"context": retriever | format_docs, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    | RunnableLambda(format_response)
)

### 9. Query Test

In [21]:
response = rag_chain.invoke("스쿼트 운동할 때, 무릎이 모이면 안되는 이유는?")
print(response)

스쿼트 운동 시 무릎이 모이는 것은 여러 가지 이유로 바람직하지 않습니다.
그 이유는 다음과 같습니다:

1.
**관절 안정성**: 무릎이 모이면 무릎 관절의 정렬이 비정상적으로 변하게 됩니다.
이는 관절의 안정성을 저하시켜 부상의 위험을 증가시킵니다.
특히, 무릎이 안쪽으로 모이는 경우에는 내측 인대에 과도한 압력이 가해져 염좌나 손상의 원인이 될 수 있습니다.
2.
**부하 분산**: 스쿼트는 하체 근육을 강화하는 운동으로, 대퇴사두근, 햄스트링, 둔근 등 다양한 근육이 협력하여 작용합니다.
무릎이 모이면 이러한 근육들이 고르게 작용하지 못하고 특정 부위에 과도한 부하가 걸리게 됩니다.
이는 근육 불균형을 초래하고, 장기적으로는 관절에 부담을 줄 수 있습니다.
3.
**운동 효율성**: 무릎이 모인 자세는 스쿼트의 운동 효율성을 떨어뜨립니다.
올바른 자세에서는 무릎이 발끝과 일직선을 이루어야 하며, 이를 통해 하체 근육이 최적으로 작용할 수 있습니다.
무릎이 모이면 운동의 효과가 감소하고, 원하는 근육을 제대로 자극하지 못하게 됩니다.
4.
**부상 예방**: 무릎이 모인 자세는 부상의 위험을 증가시킵니다.
특히, 무릎이 안쪽으로 모이는 경우에는 슬개골(무릎뼈)의 위치가 비정상적으로 변하여 슬개골 통증이나 연골 손상 등의 문제가 발생할 수 있습니다.
따라서 스쿼트를 수행할 때는 무릎이 발끝과 일직선을 이루도록 하고, 발의 위치와 무릎의 정렬을 신경 써야 합니다.
이를 통해 안전하고 효과적인 운동을 할 수 있습니다.
만약 무릎이 모이는 경향이 있다면, 모빌리티 훈련이나 적절한 스트레칭을 통해 유연성을 개선하고, 올바른 자세를 유지하는 것이 중요합니다.
